In [ ]:
anime_2=anime.copy()

In [ ]:
import re
def text_cleaning(text):
    text = re.sub(r'&quot;', '', text)
    text = re.sub(r'.hack//', '', text)
    text = re.sub(r'&#039;', '', text)
    text = re.sub(r'A&#039;s', '', text)
    text = re.sub(r'I&#039;', 'I\'', text)
    text = re.sub(r'&amp;', 'and', text)
    
    return text

anime_2['name'] = anime_2['name'].apply(text_cleaning)

In [ ]:
# Filling NaNs with empty string
anime_2['genre'] = anime_2['genre'].fillna('')
genres_str = anime_2['genre'].str.split(',').astype(str)
print(genres_str)

0        ['Drama', ' Romance', ' School', ' Supernatural']
1        ['Action', ' Adventure', ' Drama', ' Fantasy',...
2        ['Action', ' Comedy', ' Historical', ' Parody'...
3                                  ['Sci-Fi', ' Thriller']
4        ['Action', ' Comedy', ' Historical', ' Parody'...
                               ...                        
12289                                           ['Hentai']
12290                                           ['Hentai']
12291                                           ['Hentai']
12292                                           ['Hentai']
12293                                           ['Hentai']
Name: genre, Length: 12017, dtype: object


In [ ]:
genres_str.nunique()

3229

In [ ]:
genres_str.shape

(12017,)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#getting tfidf
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')
tfv_matrix = tfv.fit_transform(genres_str)
tfv_matrix

<12017x1547 sparse matrix of type '<class 'numpy.float64'>'
	with 84086 stored elements in Compressed Sparse Row format>

In [ ]:
print(tfv_matrix)
print(tfv_matrix.shape)

  (0, 1400)	0.5020288510096058
  (0, 675)	0.41414988282503656
  (0, 1461)	0.43548991186747454
  (0, 1392)	0.2989195510536398
  (0, 674)	0.33486984351736193
  (0, 1540)	0.23481052665683189
  (0, 1436)	0.2239628734799417
  (0, 1389)	0.21046591813951537
  (0, 584)	0.18898010225320216
  (1, 1152)	0.38152102324269277
  (1, 830)	0.366908014547943
  (1, 596)	0.2870714302120901
  (1, 201)	0.2639102464014669
  (1, 5)	0.2564772677373269
  (1, 1266)	0.3310053163074712
  (1, 1150)	0.3310053163074712
  (1, 827)	0.20270167599094568
  (1, 590)	0.20312816116448168
  (1, 200)	0.20909101583394968
  (1, 1)	0.1573632286168298
  (1, 1506)	0.12721002990417352
  (1, 1242)	0.1894017213460727
  (1, 1144)	0.1640248108341495
  (1, 779)	0.11633137377606641
  (1, 169)	0.11492167796706783
  :	:
  (12001, 988)	0.5451456836618241
  (12001, 411)	0.4588466015333528
  (12001, 976)	0.22641402149443268
  (12001, 1330)	0.2959481029101293
  (12001, 327)	0.13242508495662159
  (12002, 976)	1.0
  (12003, 976)	1.0
  (12004, 411

In [ ]:
cos_sim = cosine_similarity(tfv_matrix)
cos_sim.shape

(12017, 12017)

In [ ]:
cos_sim

array([[1.        , 0.02301622, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.02301622, 1.        , 0.0253986 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.0253986 , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        1.        ]])

In [ ]:
    indices2 = pd.Series(anime_2.index, index = anime_2["name"])

In [ ]:
    indices2 = indices2[~indices2.index.duplicated(keep = "last")]
    indices2

name
Kimi no Na wa.                                                   0
Fullmetal Alchemist: Brotherhood                                 1
Gintama°                                                         2
Steins;Gate                                                      3
Haikyuu!!: Karasuno Koukou VS Shiratorizawa Gakuen Koukou        5
                                                             ...  
Toushindai My Lover: Minami tai Mecha-Minami                 12289
Under World                                                  12290
Violence Gekiga David no Hoshi                               12291
Violence Gekiga Shin David no Hoshi: Inma Densetsu           12292
Yasuji no Pornorama: Yacchimae!!                             12293
Length: 12010, dtype: int64

In [ ]:
def content_based_recommender(df, movie_name = "Cowboy Bebop" , rec_count = 5):
    movie_index = indices2[movie_name]
    similarity_score = pd.DataFrame(cos_sim[movie_index], columns = ["score"])
    similar_animes = similarity_score.sort_values(by = "score", ascending = False). \
            iloc[1:rec_count].index
    return df["name"].iloc[similar_animes]

In [ ]:
def anime_searcher(df, name_words_contain):
    print(df.loc[df["name"].str.contains(name_words_contain, na = False), "name"])

In [ ]:
content_based_recommender(anime_2, movie_name = "Lupin III: $1 Money Wars", rec_count = 5)

2991    Lupin III: Princess of the Breeze - Kakusareta...
5716                                  Duel Masters Charge
1569                                  Lupin III: Part III
5717                                 Duel Masters Victory
Name: name, dtype: object

In [ ]:
anime_searcher(anime_2, name_words_contain = "Lupin")

307                        Lupin III: Cagliostro no Shiro
472              Lupin III vs. Detective Conan: The Movie
561                                      Lupin III (2015)
629                                    Lupin III: Part II
662              Lupin the IIIrd: Jigen Daisuke no Bohyou
695                         Lupin III vs. Detective Conan
838               Lupin the Third: Mine Fujiko to Iu Onna
892                    Lupin III: Episode 0 First Contact
936                                             Lupin III
1423             Lupin III: Otakara Henkyaku Daisakusen!!
1569                                  Lupin III: Part III
1633                     Lupin III: Kutabare! Nostradamus
1672                              Lupin III: Walther P-38
2009                   Lupin III: Fuuma Ichizoku no Inbou
2116           Lupin III: Chi no Kokuin - Eien no Mermaid
2153                            Lupin III (2015) Specials
2154             Lupin III: Honoo no Kioku - Tokyo Crisis
2194          